This notebook is based on the tensorflow docs: 


https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams

adapted for a regression problem.

In [1]:
%load_ext tensorboard
# Clear any logs from previous runs
#!del -rf "./logs/"

In [2]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import numpy as np
from sklearn.preprocessing import StandardScaler

In [3]:
a = tuple(range(1, 6))
type(a)
a

(1, 2, 3, 4, 5)

In [4]:
file = "https://raw.githubusercontent.com/la-escamilla/codigos/master/rho_tanh15_phy_rho_corr%2BCBAO%2BHD%2BSN_nested_dynesty_multi_1.txt"

split = 0.8
epochs = 500

cols = tuple(range(1, 19))
data = np.loadtxt(file, usecols=cols)
params = data[:, 1:]
logL = data[:, 0]


nparams = len(params)
randomize = np.random.permutation(nparams)
params = params[randomize]
logL = logL[randomize]

ntrain = int(split * nparams)
indx = [ntrain]

params_training, params_testing = np.split(params, indx)
logL_training, logL_testing = np.split(logL, indx)


 # create scaler
scaler = StandardScaler()
 # # scaler = MinMaxScaler(feature_range=(-1,1))
 # # fit scaler on data
scaler.fit(params_training)
 # # apply transform
params_training = scaler.transform(params_training)
params_testing = scaler.transform(params_testing)

print("Total len dataset {}," 
      "len training set {}, len test set {}".format(nparams, 
                                                 len(params_training), len(params_testing)))



Total len dataset 40039,len training set 32031, len test set 8008


In [5]:
# loss -> val_loss
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min',
                                   min_delta=0,
                                   patience=10,
                                   restore_best_weights=True)
                                   ]

n_cols = params_training.shape[1]

In [6]:
HP_NUM_UNITS1 = hp.HParam('num_units1', hp.Discrete([100, 150, 200]))
HP_NUM_UNITS2 = hp.HParam('num_units2', hp.Discrete([100, 150, 200]))
HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([100, 150, 200]))
HP_NUM_UNITS4 = hp.HParam('num_units4', hp.Discrete([100, 150, 200]))


HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([128]))

In [7]:
# METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
      hp.hparams_config(
        hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_NUM_UNITS4, HP_BATCHSIZE],
        metrics=[hp.Metric('loss', display_name="Loss")])


In [8]:
def train_test_model(hparams):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS1], activation=tf.nn.relu, input_shape=(n_cols,)),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS2], activation=tf.nn.relu),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS3], activation=tf.nn.relu),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS4], activation=tf.nn.relu),
        tf.keras.layers.Dense(1)
    ])
                              
    model.compile(
            optimizer='adam',
            loss='mean_squared_error',
            metrics=['mean_squared_error']
          )

    
    model.fit(params_training, logL_training, epochs=epochs, 
              batch_size=hparams[HP_BATCHSIZE], callbacks=callbacks) 
    _, loss = model.evaluate(params_testing, logL_testing)


    return loss
    

In [9]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        loss = train_test_model(hparams)
        tf.summary.scalar("loss", loss, step=1)

In [10]:
session_num = 0

for batch_size in HP_BATCHSIZE.domain.values:
    for num_units1 in HP_NUM_UNITS1.domain.values:
        for num_units2 in HP_NUM_UNITS2.domain.values:
            for num_units3 in HP_NUM_UNITS3.domain.values:
                for num_units4 in HP_NUM_UNITS4.domain.values:
                    hparams = {
                        HP_NUM_UNITS1: num_units1,
                        HP_NUM_UNITS2: num_units2,
                        HP_NUM_UNITS3: num_units3,
                        HP_NUM_UNITS4: num_units4,
                        HP_BATCHSIZE: batch_size
                    }
                    run_name = "run-%d" % session_num
                    print('--- Starting trial: %s' % run_name)
                    print({h.name: hparams[h] for h in hparams})
                    run('logs/hparam_tuning/' + run_name, hparams)
                    session_num += 1

                    

--- Starting trial: run-0
{'num_units1': 100, 'num_units2': 100, 'num_units3': 100, 'num_units4': 100, 'batch_size': 128}
Train on 32031 samples
Epoch 1/500
32031/32031 [==============================] - 1s 22us/sample - loss: 616160.0872 - mean_squared_error: 616160.0000
Epoch 2/500
32031/32031 [==============================] - 0s 12us/sample - loss: 113338.1199 - mean_squared_error: 113338.1172
Epoch 3/500
32031/32031 [==============================] - 0s 11us/sample - loss: 18999.1074 - mean_squared_error: 18999.1055
Epoch 4/500
32031/32031 [==============================] - 0s 12us/sample - loss: 13239.0233 - mean_squared_error: 13239.0225
Epoch 5/500
32031/32031 [==============================] - 0s 12us/sample - loss: 10829.7622 - mean_squared_error: 10829.7607
Epoch 6/500
32031/32031 [==============================] - 0s 12us/sample - loss: 9849.1739 - mean_squared_error: 9849.1777
Epoch 7/500
32031/32031 [==============================] - 0s 12us/sample - loss: 8747.2952 - mea

32031/32031 [==============================] - 0s 11us/sample - loss: 2538.7744 - mean_squared_error: 2538.7732
Epoch 66/500
32031/32031 [==============================] - 0s 12us/sample - loss: 2357.7733 - mean_squared_error: 2357.7737
Epoch 67/500
32031/32031 [==============================] - 0s 11us/sample - loss: 2533.9582 - mean_squared_error: 2533.9585
Epoch 68/500
32031/32031 [==============================] - 0s 11us/sample - loss: 2198.7817 - mean_squared_error: 2198.7812
Epoch 69/500
32031/32031 [==============================] - 0s 11us/sample - loss: 2362.4854 - mean_squared_error: 2362.4849
Epoch 70/500
32031/32031 [==============================] - 0s 12us/sample - loss: 2571.1317 - mean_squared_error: 2571.1323
Epoch 71/500
8008/8008 [==============================] - 0s 25us/sample - loss: 7641.4865 - mean_squared_error: 7641.4854
--- Starting trial: run-1
{'num_units1': 100, 'num_units2': 100, 'num_units3': 100, 'num_units4': 150, 'batch_size': 128}
Train on 32031 sam

32031/32031 [==============================] - 0s 12us/sample - loss: 2420.6003 - mean_squared_error: 2420.6008
Epoch 58/500
32031/32031 [==============================] - 0s 12us/sample - loss: 2349.2171 - mean_squared_error: 2349.2170
Epoch 59/500
32031/32031 [==============================] - 0s 12us/sample - loss: 2929.6363 - mean_squared_error: 2929.6357
Epoch 60/500
32031/32031 [==============================] - 0s 12us/sample - loss: 2855.9943 - mean_squared_error: 2855.9954
Epoch 61/500
32031/32031 [==============================] - 0s 12us/sample - loss: 3631.8545 - mean_squared_error: 3631.8555
Epoch 62/500
32031/32031 [==============================] - 0s 12us/sample - loss: 2571.0392 - mean_squared_error: 2571.0405
Epoch 63/500
32031/32031 [==============================] - 0s 12us/sample - loss: 2277.6353 - mean_squared_error: 2277.6350
Epoch 64/500
32031/32031 [==============================] - 0s 12us/sample - loss: 2472.7611 - mean_squared_error: 2472.7615
Epoch 65/500


32031/32031 [==============================] - 0s 12us/sample - loss: 3904.6549 - mean_squared_error: 3904.6543
Epoch 25/500
32031/32031 [==============================] - 0s 13us/sample - loss: 3863.1246 - mean_squared_error: 3863.1248
Epoch 26/500
32031/32031 [==============================] - 0s 13us/sample - loss: 4568.0181 - mean_squared_error: 4568.0200
Epoch 27/500
32031/32031 [==============================] - 0s 12us/sample - loss: 3856.4529 - mean_squared_error: 3856.4529
Epoch 28/500
32031/32031 [==============================] - 0s 13us/sample - loss: 4052.0092 - mean_squared_error: 4052.0083
Epoch 29/500
32031/32031 [==============================] - 0s 13us/sample - loss: 3935.0319 - mean_squared_error: 3935.0325
Epoch 30/500
32031/32031 [==============================] - 0s 12us/sample - loss: 3733.4175 - mean_squared_error: 3733.4165
Epoch 31/500
32031/32031 [==============================] - 0s 13us/sample - loss: 3685.8904 - mean_squared_error: 3685.8899
Epoch 32/500


32031/32031 [==============================] - 0s 12us/sample - loss: 4004.6190 - mean_squared_error: 4004.6184
Epoch 25/500
32031/32031 [==============================] - 0s 13us/sample - loss: 3918.0313 - mean_squared_error: 3918.0327
Epoch 26/500
32031/32031 [==============================] - 0s 12us/sample - loss: 4140.1332 - mean_squared_error: 4140.1333
Epoch 27/500
32031/32031 [==============================] - 0s 12us/sample - loss: 3815.3441 - mean_squared_error: 3815.3430
Epoch 28/500
32031/32031 [==============================] - 0s 12us/sample - loss: 3491.5729 - mean_squared_error: 3491.5742
Epoch 29/500
32031/32031 [==============================] - 0s 12us/sample - loss: 3723.6818 - mean_squared_error: 3723.6829
Epoch 30/500
32031/32031 [==============================] - 0s 12us/sample - loss: 3624.5759 - mean_squared_error: 3624.5767
Epoch 31/500
32031/32031 [==============================] - 0s 12us/sample - loss: 3898.7222 - mean_squared_error: 3898.7222
Epoch 32/500


...

32031/32031 [==============================] - 1s 17us/sample - loss: 2384.2136 - mean_squared_error: 2384.2141
Epoch 52/500
32031/32031 [==============================] - 1s 25us/sample - loss: 2052.4443 - mean_squared_error: 2052.4438
Epoch 53/500
32031/32031 [==============================] - 1s 18us/sample - loss: 2892.8230 - mean_squared_error: 2892.8242
Epoch 54/500
32031/32031 [==============================] - 1s 16us/sample - loss: 3633.5747 - mean_squared_error: 3633.5750
Epoch 55/500
32031/32031 [==============================] - 0s 16us/sample - loss: 3306.7986 - mean_squared_error: 3306.7996
Epoch 56/500
32031/32031 [==============================] - 1s 22us/sample - loss: 3188.8993 - mean_squared_error: 3188.8987
Epoch 57/500
32031/32031 [==============================] - 1s 20us/sample - loss: 2199.9147 - mean_squared_error: 2199.9148
Epoch 58/500
32031/32031 [==============================] - 1s 21us/sample - loss: 1874.2822 - mean_squared_error: 1874.2821
Epoch 59/500


32031/32031 [==============================] - 1s 19us/sample - loss: 2269.5415 - mean_squared_error: 2269.5408
Epoch 47/500
32031/32031 [==============================] - 1s 22us/sample - loss: 2488.3403 - mean_squared_error: 2488.3416
Epoch 48/500
32031/32031 [==============================] - 1s 19us/sample - loss: 2135.9106 - mean_squared_error: 2135.9111
Epoch 49/500
32031/32031 [==============================] - 1s 17us/sample - loss: 2817.6048 - mean_squared_error: 2817.6052
Epoch 50/500
32031/32031 [==============================] - 1s 17us/sample - loss: 2470.3216 - mean_squared_error: 2470.3193
Epoch 51/500
32031/32031 [==============================] - 1s 16us/sample - loss: 2493.3215 - mean_squared_error: 2493.3215
Epoch 52/500
32031/32031 [==============================] - 1s 16us/sample - loss: 2145.3683 - mean_squared_error: 2145.3684
Epoch 53/500
32031/32031 [==============================] - 1s 17us/sample - loss: 2202.0267 - mean_squared_error: 2202.0264
Epoch 54/500


32031/32031 [==============================] - 1s 18us/sample - loss: 3634.1833 - mean_squared_error: 3634.1831
Epoch 52/500
32031/32031 [==============================] - 1s 16us/sample - loss: 2538.6473 - mean_squared_error: 2538.6475
Epoch 53/500
32031/32031 [==============================] - 1s 16us/sample - loss: 1925.1812 - mean_squared_error: 1925.1809
Epoch 54/500
32031/32031 [==============================] - 1s 17us/sample - loss: 2426.1959 - mean_squared_error: 2426.1953
Epoch 55/500
32031/32031 [==============================] - 1s 19us/sample - loss: 2133.7578 - mean_squared_error: 2133.7581
Epoch 56/500
32031/32031 [==============================] - 1s 17us/sample - loss: 2055.1639 - mean_squared_error: 2055.1633
Epoch 57/500
32031/32031 [==============================] - 1s 21us/sample - loss: 2325.7948 - mean_squared_error: 2325.7944
Epoch 58/500
32031/32031 [==============================] - 1s 17us/sample - loss: 3767.3891 - mean_squared_error: 3767.3884
Epoch 59/500


32031/32031 [==============================] - 1s 18us/sample - loss: 3610.8861 - mean_squared_error: 3610.8877
Epoch 39/500
32031/32031 [==============================] - 1s 16us/sample - loss: 3126.5113 - mean_squared_error: 3126.5115
Epoch 40/500
32031/32031 [==============================] - 1s 17us/sample - loss: 2467.2769 - mean_squared_error: 2467.2776
Epoch 41/500
32031/32031 [==============================] - 1s 16us/sample - loss: 3124.6445 - mean_squared_error: 3124.6431
Epoch 42/500
32031/32031 [==============================] - 1s 16us/sample - loss: 2800.1830 - mean_squared_error: 2800.1821
Epoch 43/500
32031/32031 [==============================] - 1s 16us/sample - loss: 2319.7823 - mean_squared_error: 2319.7815
Epoch 44/500
32031/32031 [==============================] - 1s 17us/sample - loss: 2148.2553 - mean_squared_error: 2148.2549
Epoch 45/500
32031/32031 [==============================] - 1s 16us/sample - loss: 2360.8682 - mean_squared_error: 2360.8684
Epoch 46/500


8008/8008 [==============================] - 0s 56us/sample - loss: 6979.1675 - mean_squared_error: 6979.1665
--- Starting trial: run-80
{'num_units1': 200, 'num_units2': 200, 'num_units3': 200, 'num_units4': 200, 'batch_size': 128}
Train on 32031 samples
Epoch 1/500
32031/32031 [==============================] - 1s 40us/sample - loss: 476011.4555 - mean_squared_error: 476011.6250
Epoch 2/500
32031/32031 [==============================] - 1s 26us/sample - loss: 25959.3713 - mean_squared_error: 25959.3730
Epoch 3/500
32031/32031 [==============================] - 1s 25us/sample - loss: 12268.2589 - mean_squared_error: 12268.2588
Epoch 4/500
32031/32031 [==============================] - 1s 29us/sample - loss: 9718.7703 - mean_squared_error: 9718.7695
Epoch 5/500
32031/32031 [==============================] - 1s 27us/sample - loss: 10803.8509 - mean_squared_error: 10803.8545
Epoch 6/500
32031/32031 [==============================] - 1s 27us/sample - loss: 7218.6968 - mean_squared_error: 

In [12]:
%tensorboard --logdir logs/hparam_tuning

Reusing TensorBoard on port 6006 (pid 8720), started 0:13:19 ago. (Use '!kill 8720' to kill it.)